Fine tuning sentence-transformers/all-MiniLM-L12-v2 with  enron email dataset and LoRa with peft


In [ ]:
%pip install transformers datasets torch peft pandas scikit-learn safetensors sentence-transformers
%pip install "numpy<2.0.0"
%pip install kaggle kagglehub
%pip install seaborn matplotlib


In [2]:
import pandas as pd
import torch
import re
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
import torch.nn as nn
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments
from datasets import Dataset, Features, Value, ClassLabel
import safetensors # Required for use_safetensors=True and safe_serialization=True
from peft import LoraConfig, get_peft_model
import kagglehub

# Force CPU
torch.cuda.is_available = lambda: False
device = torch.device("cpu")

In [3]:
# download dataset, kaggle will download to cache 
path = kagglehub.dataset_download("wcukierski/enron-email-dataset")
print("Path to dataset files:", path)

100%|██████████| 358M/358M [03:59<00:00, 1.56MB/s] 

Extracting files...


Path to dataset files: /Users/rodneyfinkel/.cache/kagglehub/datasets/wcukierski/enron-email-dataset/versions/2


In [4]:
# move kaggle dataset to current working directory
import shutil
import os

for filename in os.listdir(path):
    src = os.path.join(path, filename)
    dst = os.path.join(".", filename)
    if os.path.isfile(src):
        shutil.copy(src, dst)
        print(f"copied {filename} to current working directory.")
        # Remove the orginal cached dataset
        cache_path = os.path.expanduser("~/.cache/kagglehub/datasets/wcukierski/enron-email-dataset")
        shutil.rmtree(cache_path)
        print("Original cached dataset removed.")
        

copied emails.csv to current working directory.
Original cached dataset removed.


In [5]:
# Load and preprocess dataset
df = pd.read_csv('emails.csv')
df = df.sample(n=20000, random_state=42)

In [6]:
pd.set_option('display.max_columns', None)
df.head()

,file,message
427616,shackleton-s/sent/1912.,Message-ID: <21013688.1075844564560.JavaMail.e...
108773,farmer-d/logistics/1066.,Message-ID: <22688499.1075854130303.JavaMail.e...
355471,parks-j/deleted_items/202.,Message-ID: <27817771.1075841359502.JavaMail.e...
457837,stokley-c/chris_stokley/iso/client_rep/41.,Message-ID: <10695160.1075858510449.JavaMail.e...
124910,germany-c/all_documents/1174.,Message-ID: <27819143.1075853689038.JavaMail.e...


In [7]:
# Clean emails and extract subject and delete dates
def clean_email(text):
    # extract subject if available
    subject_match = re.search(r'Subject: (.*?)\n', text, re.IGNORECASE)
    subject = subject_match.group(1) if subject_match else ''
    # Remove headers, signatures and dates
    text = re.sub(r'From:.*\n|To:.*\n|Subject:.*\n|Message-ID:.*\n|Date:.*\n', '', text)
    text = re.sub(r'-{2,}.*?-{2,}', '', text)
    text = re.sub(r'http[s]?://\S+', '', text)
    return text.strip(), subject

df['message'], df['subject'] = zip(*df['message'].apply(clean_email))


In [8]:
df.head()

,file,message,subject
427616,shackleton-s/sent/1912.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Re: Credit Derivatives
108773,farmer-d/logistics/1066.,Cc: daren.farmer@enron.com\nMime-Version: 1.0\...,Meter #1591 Lamay Gaslift
355471,parks-j/deleted_items/202.,"wollam.erik@enron.com, corrier.brad@enron.com\...",Re: man night again?
457837,stokley-c/chris_stokley/iso/client_rep/41.,Mime-Version: 1.0\nContent-Type: text/plain; c...,"Enron 480, 1480 charges"
124910,germany-c/all_documents/1174.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Transport Deal


In [9]:
# Heuristic spam labeling
def is_potential_spam(text, subject):
    spam_keywords = ['offer', 'free', 'win', 'click here', 'urgent', 'limited time', 'act now']
    return any(keyword in text.lower() or keyword in subject.lower() for keyword in spam_keywords)

df['label'] = df.apply(lambda x: is_potential_spam(x['message'], x['subject']), axis=1).astype(int)

spam = df[df['label'] == 1]
non_spam = df[df['label'] == 0]
target_spam_count = int(len(df) * 0.05)

if len(spam) > target_spam_count:
    print(f"Reducing spam emails from {len(spam)} to {target_spam_count}")
    spam = resample(spam, n_samples=target_spam_count, random_state=42)
else:
    non_spam = resample(non_spam, n_samples=int(target_spam_count * 3), random_state=42)

df = pd.concat([spam, non_spam]).sample(frac=1, random_state=42).reset_index(drop=True)
df.drop(columns=['label'], inplace=True)

Reducing spam emails from 6549 to 1000


In [10]:
print(df.shape)
df.head()

(14451, 3)


,file,message,subject
0,sanders-r/all_documents/3872.,"Cc: linda.guinn@enron.com, showard@milbank.com...",Re: NSM Docs.
1,haedicke-m/notes_inbox/359.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Fw: Your Committee Minutes on Equity & Energy ...
2,scott-s/gir/164.,"johnj@bcjlaw.com, sscott3@enron.com, ghinners@...",Fwd: Exhibit 111
3,derrick-j/inbox/200.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Hello from Up State New York
4,kean-s/sent_items/315.,Mime-Version: 1.0\nContent-Type: text/plain; c...,RE: all employee meeting


In [11]:
# Heuristic category labeling
def assign_category(text, subject, file):
    text = text.lower()
    subject = subject.lower()
    file = file.lower() 
    if any(kw in text or kw in subject for kw in ['budget', 'invoice', 'payment', 'financial']) or 'finance' in file or 'accounting' in file:
        return 0  # Finance
    elif any(kw in text or kw in subject for kw in ['hiring', 'employee', 'benefits', 'payroll']) or 'hr' in file or 'personnel' in file:
        return 1  # HR
    elif any(kw in text or kw in subject for kw in ['contract', 'legal', 'compliance', 'attorney']) or 'legal' in file:
        return 2  # Legal
    elif any(kw in text or kw in subject for kw in ['meeting', 'schedule', 'calendar', 'agenda']) or 'meetings' in file or 'admin' in file:
        return 3  # Admin
    return 0  # Default to finance (most common in Enron)

df['category'] = df.apply(lambda x: assign_category(x['message'], x['subject'], x['file']), axis=1)

In [12]:
df.head()

,file,message,subject,category
0,sanders-r/all_documents/3872.,"Cc: linda.guinn@enron.com, showard@milbank.com...",Re: NSM Docs.,1
1,haedicke-m/notes_inbox/359.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Fw: Your Committee Minutes on Equity & Energy ...,3
2,scott-s/gir/164.,"johnj@bcjlaw.com, sscott3@enron.com, ghinners@...",Fwd: Exhibit 111,0
3,derrick-j/inbox/200.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Hello from Up State New York,0
4,kean-s/sent_items/315.,Mime-Version: 1.0\nContent-Type: text/plain; c...,RE: all employee meeting,0


In [13]:
# Heuristic priority labeling
def assign_priority(text, subject):
    text = text.lower()
    subject = subject.lower()
    if any(kw in text or kw in subject for kw in ['urgent', 'asap', 'deadline', 'immediate', 'action required']) or '!' in text:
        return 0  # High
    elif any(kw in text or kw in subject for kw in ['fyi', 'thanks', 'no rush']):
        return 2  # Low
    return 1  # Medium

df['priority'] = df.apply(lambda x: assign_priority(x['message'], x['subject']), axis=1)

In [14]:
df.head()

,file,message,subject,category,priority
0,sanders-r/all_documents/3872.,"Cc: linda.guinn@enron.com, showard@milbank.com...",Re: NSM Docs.,1,0
1,haedicke-m/notes_inbox/359.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Fw: Your Committee Minutes on Equity & Energy ...,3,2
2,scott-s/gir/164.,"johnj@bcjlaw.com, sscott3@enron.com, ghinners@...",Fwd: Exhibit 111,0,2
3,derrick-j/inbox/200.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Hello from Up State New York,0,1
4,kean-s/sent_items/315.,Mime-Version: 1.0\nContent-Type: text/plain; c...,RE: all employee meeting,0,1


In [15]:
# Balance categories (~25% each)
category_counts = df['category'].value_counts()
target_count = int(len(df) * 0.25)
df_cat_balanced = pd.DataFrame()
for cat in range(4):  # 0=finance, 1=hr, 2=legal, 3=admin
    cat_df = df[df['category'] == cat]
    if len(cat_df) > target_count:
        cat_df = resample(cat_df, n_samples=target_count, random_state=42)
    df_cat_balanced = pd.concat([df_cat_balanced, cat_df])

In [16]:
# Balance priorities (~33% each)
priority_counts = df['priority'].value_counts()
target_count = int(len(df) * 0.33)
df_prio_balanced = pd.DataFrame()
for prio in range(3):  # 0=high, 1=medium, 2=low
    prio_df = df[df['priority'] == prio]
    if len(prio_df) > target_count:
        prio_df = resample(prio_df, n_samples=target_count, random_state=42)
    df_prio_balanced = pd.concat([df_prio_balanced, prio_df])

In [17]:
print(df.shape)
df.head()

(14451, 5)


,file,message,subject,category,priority
0,sanders-r/all_documents/3872.,"Cc: linda.guinn@enron.com, showard@milbank.com...",Re: NSM Docs.,1,0
1,haedicke-m/notes_inbox/359.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Fw: Your Committee Minutes on Equity & Energy ...,3,2
2,scott-s/gir/164.,"johnj@bcjlaw.com, sscott3@enron.com, ghinners@...",Fwd: Exhibit 111,0,2
3,derrick-j/inbox/200.,Mime-Version: 1.0\nContent-Type: text/plain; c...,Hello from Up State New York,0,1
4,kean-s/sent_items/315.,Mime-Version: 1.0\nContent-Type: text/plain; c...,RE: all employee meeting,0,1


In [ ]:
# Train model function
def train_model(dataset, model_name, num_labels, output_dir, label_column, label_names):
    # Combine subject and message for training
    dataset['text'] = dataset.apply(lambda x: f"[SUBJECT] {x['subject']} [BODY] {x['message']}", axis=1)
    train_texts, val_texts, train_labels, val_labels = train_test_split(
        dataset['text'], dataset[label_column], test_size=0.2, random_state=42
    )
    
     # Verify labels are integers
    train_labels = train_labels.astype(int)
    val_labels = val_labels.astype(int)
    
    # Define dataset features with ClassLabel
    features = Features({
        'text': Value('string'),
        'label': ClassLabel(num_classes=num_labels, names=label_names)
    })
    
    train_dataset = Dataset.from_dict({'text': train_texts, 'label': train_labels}, features=features)
    val_dataset = Dataset.from_dict({'text': val_texts, 'label': val_labels}, features=features)
    
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    
    # Define id2label and label2id
    id2label = {i: label for i, label in enumerate(label_names)}
    label2id = {label: i for i, label in enumerate(label_names)}    
    
    # changed to use_safetensors = True
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=num_labels, id2label=id2label, label2id=label2id, use_safetensors=True)
    
    # Ensure model is in training mode and freeze base model parameters
    model.train()
    for param in model.parameters():
        param.requires_grad_(False) # Freeze base model
    lora_config = LoraConfig(r=8, lora_alpha=16, target_modules=[
        'query', 'key', 'value',  # Attention layers
        'dense',  # Feed-forward layers
        'classifier'  # Sequence classification head
        
        ], 
        lora_dropout=0.1, 
        bias="none", 
        task_type="SEQ_CLS",
        modules_to_save=['classifier']  # Ensure classifier is saved
        ) 
    model = get_peft_model(model, lora_config)
    
    # Enable gradient for both LoRA and classifier parameters
    trainable_params = []
    for name, param in model.named_parameters():
        if any(substr in name for substr in ["lora", "classifier"]):
            param.requires_grad = True
            trainable_params.append(name)
    
    # Debug: Print trainable parameters to verify
    print(f"Trainable parameters: {trainable_params}")
    
    def tokenize_function(examples):
        return tokenizer(examples['text'], padding='max_length', truncation=True, max_length=128) # May need to lower max_length to 64 becuase of memory
    
    train_dataset = train_dataset.map(tokenize_function, batched=True)
    val_dataset = val_dataset.map(tokenize_function, batched=True)
    
    train_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])
    val_dataset.set_format('torch', columns=['input_ids', 'attention_mask', 'label'])  
    
    # Evaluation metrics
    def compute_metrics(p):
        preds = p.predictions.argmax(axis=-1)
        labels = p.label_ids
        return {
            "accuracy": accuracy_score(labels, preds),
            "f1": f1_score(labels, preds, average="weighted")
        }
    
    training_args = TrainingArguments(
        output_dir=output_dir,
        num_train_epochs=3,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        warmup_steps=50,
        weight_decay=0.01,
        logging_dir=f'{output_dir}/logs',
        logging_steps=10,
        eval_strategy='steps',
        save_strategy='steps',
        save_steps=200,
        gradient_checkpointing=False, # set to False since LoRA handles memory optimization
        fp16=False,
        load_best_model_at_end=True,
    )
    
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        compute_metrics=compute_metrics
    )
    
     # Debug: Print training dataset features
    print(f"Training dataset features: {train_dataset.features}")
    
    # Train and evaluate
    trainer.train()
    metrics = trainer.evaluate()
    print(f"Evaluation metrics for {output_dir}: {metrics}")
    
    # Confusion Matrix
    preds = trainer.predict(val_dataset)
    cm = confusion_matrix(preds.label_ids, preds.predictions.argmax(axis=1))
    plt.figure(figsize=(8, 6))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=label_names, yticklabels=label_names)
    plt.title(f'Confusion Matrix - {output_dir}')
    plt.xlabel('Predicted')
    plt.ylabel('True')
    plt.show()
    
    model.save_pretrained(output_dir, safe_serialization=True)
    tokenizer.save_pretrained(output_dir)
    return model, tokenizer

In [ ]:
# Train category classifier
# changed model to sentence-transformers/all-MiniLM-L12-v2 from microsoft/minilm-l12-h384-uncased
category_label_names = ["Finance", "HR", "Legal", "Admin"]
model_cat, tokenizer_cat = train_model(
    df_cat_balanced, 'sentence-transformers/all-MiniLM-L12-v2', 4, './fine_tuned_minilm_category', 'category', category_label_names) 

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['base_model.model.bert.encoder.layer.0.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.key.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.key.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.value.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.output.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.output.dense.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.intermediate.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.intermediate.dense.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.output.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.output.dense.lora_B.defaul

Map:   0%|          | 0/7401 [00:00<?, ? examples/s]

Map:   0%|          | 0/1851 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Training dataset features: {'text': Value('string'), 'label': ClassLabel(names=['Finance', 'HR', 'Legal', 'Admin']), 'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8'))}


Step,Training Loss,Validation Loss,Accuracy,F1
10,1.390900,1.386376,0.244192,0.164412
20,1.383500,1.385700,0.260940,0.187482
30,1.387100,1.385150,0.273906,0.203970
40,1.383100,1.383995,0.306861,0.239736
50,1.375000,1.381259,0.392761,0.265216
60,1.383300,1.377555,0.391680,0.220472
70,1.373500,1.374453,0.391680,0.220472
80,1.371800,1.370963,0.391680,0.220472
90,1.380300,1.368014,0.391680,0.220472
100,1.364100,1.364220,0.391680,0.220472


/Users/rodneyfinkel/Desktop/matrix_dna_email_agent/env/lib/python3.12/site-packages/peft/utils/other.py:1221: UserWarning: Unable to fetch remote file due to the following error (ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 865b3e15-424d-448e-a295-e555d7605376)') - silently ignoring the lookup for the file config.json in sentence-transformers/all-MiniLM-L12-v2.
  warnings.warn(
/Users/rodneyfinkel/Desktop/matrix_dna_email_agent/env/lib/python3.12/site-packages/peft/utils/save_and_load.py:238: UserWarning: Could not find a config file in sentence-transformers/all-MiniLM-L12-v2 - will assume that the vocabulary was not modified.
  warnings.warn(


In [19]:
# Train priority classifier
# changed model to sentence-transformers/all-MiniLM-L12-v2 from microsoft/minilm-l12-h384-uncased
priority_label_names = ["High", "Medium", "Low"]
model_prio, tokenizer_prio = train_model(
    df_prio_balanced, 'sentence-transformers/all-MiniLM-L12-v2', 3, './fine_tuned_minilm_priority', 'priority', priority_label_names
)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at sentence-transformers/all-MiniLM-L12-v2 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Trainable parameters: ['base_model.model.bert.encoder.layer.0.attention.self.query.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.query.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.key.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.key.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.value.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.self.value.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.attention.output.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.attention.output.dense.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.intermediate.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.intermediate.dense.lora_B.default.weight', 'base_model.model.bert.encoder.layer.0.output.dense.lora_A.default.weight', 'base_model.model.bert.encoder.layer.0.output.dense.lora_B.defaul

Map:   0%|          | 0/8944 [00:00<?, ? examples/s]

Map:   0%|          | 0/2236 [00:00<?, ? examples/s]

No label_names provided for model class `PeftModelForSequenceClassification`. Since `PeftModel` hides base models input arguments, if label_names is not given, label_names can't be set automatically within `Trainer`. Note that empty label_names list will be used instead.


Training dataset features: {'text': Value('string'), 'label': ClassLabel(names=['High', 'Medium', 'Low']), 'input_ids': List(Value('int32')), 'token_type_ids': List(Value('int8')), 'attention_mask': List(Value('int8'))}


Step,Training Loss,Validation Loss,Accuracy,F1
10,1.102800,1.100014,0.273703,0.196321
20,1.099300,1.099821,0.274150,0.210412
30,1.098800,1.099716,0.275045,0.194063
40,1.097700,1.099623,0.276386,0.190018
50,1.100000,1.098868,0.309034,0.259440
60,1.099700,1.097073,0.412343,0.284342
70,1.095500,1.095820,0.418157,0.253939
80,1.100100,1.095433,0.419946,0.253808
90,1.091300,1.094063,0.422182,0.251264
100,1.095300,1.092687,0.423077,0.251559


KeyboardInterrupt: 

In [ ]:
# Inference functions
def predict_category(email_text, model=model_cat, tokenizer=tokenizer_cat):
    text, subject = clean_email(email_text)
    input_text = f"[SUBJECT] {subject} [BODY] {text}"
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return category_label_names[predicted_class]

def predict_priority(email_text, model=model_prio, tokenizer=tokenizer_prio):
    text, subject = clean_email(email_text)
    input_text = f"[SUBJECT] {subject} [BODY] {text}"
    inputs = tokenizer(input_text, return_tensors='pt', truncation=True, padding=True, max_length=64)
    with torch.no_grad():
        outputs = model(**inputs)
    predicted_class = torch.argmax(outputs.logits, dim=1).item()
    return priority_label_names[predicted_class]

# Test inference
sample_email = "Subject: Budget Review\nDate: Wed, 29 Nov 2000 05:40:00 -0800\nPlease review the attached budget for Q3."
print(f"Category: {predict_category(sample_email)}")
print(f"Priority: {predict_priority(sample_email)}")